# Implementación Naive Bayes para Seguimiento Docente

## Objetivo

Este notebook es una guía práctica que muestra, paso a paso, cómo implementar un clasificador **Naive Bayes** utilizando Python y la librería Scikit-learn. El objetivo es predecir la valoración del desempeño de un docente (`Valoracion`) basándonos en las características de los cursos que imparte, utilizando los datos de seguimiento proporcionados.

## Paso 1: Importar las Librerías Necesarias

Primero, importamos todas las herramientas que vamos a necesitar.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import warnings

warnings.filterwarnings('ignore')

## Paso 2: Cargar y Preparar los Datos

Cargamos los datos desde un archivo (`.csv`, `.xlsx`).

In [9]:
# --- ¡ACCIÓN REQUERIDA! ---
data = "data_base.xlsx"

try:
    df = pd.read_excel(data)
    print("¡Archivo cargado exitosamente!")
    print("Primeras 5 filas:")
    display(df.head())
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{data}'. Asegúrate de que esté en la misma carpeta que este notebook.")

¡Archivo cargado exitosamente!
Primeras 5 filas:


,NRC,Periodo,Asignatura,Momento,Salon,Tipo,Id_Docente,Responsable,Sede_Docente,Ubicación,Tipo_Curso,Plantilla,Total_Alistamiento,Total_Ejecucion,Total_Curso,Valoracion,Email_Enviado
0,43-300,202443,Metodología de la Investigac,3,B3-304,1,478259,AEMD,IBA,IBA,PD,INNOVAME,50.0,0.0,50.0,Desempeño insatisfactorio,-
1,43-303,202443,Formulación de Proyectos,1,B3-304,1,662326,ESP,IBA,IBA,PD,INNOVAME,30.0,36.3,46.3,Desempeño insatisfactorio,SI
2,43-302,202443,Evaluación de Proyectos,2,B2-305,1,662326,ESP,IBA,IBA,PD,INNOVAME,20.0,30.0,50.0,Desempeño insatisfactorio,-
3,43-304,202443,Gerencia del Conoc y la Tecnol,2,VIRTU,2,846904,ESP,IBA,IBA,PD,INNOVAME,50.0,35.0,85.0,Desempeño Bueno,-
4,43-56330,202443,Electiva,2,B3-304,1,738598,ESP,IBA,IBA,PD,INNOVAME,50.0,40.0,90.0,Desempeño Bueno,-


## Paso 3: Preprocesamiento de los Datos

Los modelos de Machine Learning no entienden texto. Por lo tanto, debemos convertir nuestras características categóricas (`Tipo`, `Tipo_Curso`, `Plantilla`) en representaciones numéricas. Este proceso se llama **codificación**.

1.  **Separar Características (X) y Variable Objetivo (y)**: Aislamos las columnas que usaremos para predecir (X) de la columna que queremos predecir (y).
2.  **Codificar Características**: Usaremos `OrdinalEncoder` para asignar un número entero único a cada categoría de texto.

In [10]:
# Separamos las características de la variable objetivo
X = df[['Tipo', 'Tipo_Curso', 'Plantilla']]
y = df['Valoracion']

# 1) Eliminar filas donde la variable objetivo es NaN (no se puede entrenar con y faltante)
mask = y.notna()
X = X[mask].reset_index(drop=True)
y = y[mask].reset_index(drop=True)

# 2) Imputar valores faltantes en las características con la moda (valor más frecuente)
imputer = SimpleImputer(strategy='constant', fill_value='DESCONOCIDO')
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)

# Creamos una instancia del codificador
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Ajustamos y transformamos los datos. El modelo aprende las categorías y las convierte en números.
X_encoded = encoder.fit_transform(X)

print("Características originales (X):")
print(X.head())
print("Características codificadas (listas para el modelo):")
print(X_encoded[:5])

Características originales (X):
  Tipo Tipo_Curso Plantilla
0    1         PD  INNOVAME
1    1         PD  INNOVAME
2    1         PD  INNOVAME
3    2         PD  INNOVAME
4    1         PD  INNOVAME
Características codificadas (listas para el modelo):
[[0. 0. 2.]
 [0. 0. 2.]
 [0. 0. 2.]
 [1. 0. 2.]
 [0. 0. 2.]]


## Paso 4: Dividir los Datos en Conjuntos de Entrenamiento y Prueba

Es una práctica fundamental en Machine Learning. Entrenamos el modelo con la mayoría de los datos (conjunto de entrenamiento) y luego evaluamos su rendimiento en un conjunto de datos que nunca ha visto (conjunto de prueba). Esto nos da una medida de su capacidad para generalizar.

In [11]:
# Dividimos los datos: 75% para entrenamiento, 25% para prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras')

Tamaño del conjunto de entrenamiento: 949 muestras
Tamaño del conjunto de prueba: 317 muestras


## Paso 5: Entrenar el Modelo Naive Bayes

Ahora llega el momento de crear y entrenar nuestro clasificador. Como nuestras características son categóricas, el modelo `CategoricalNB` de Scikit-learn es el más apropiado.

In [12]:
# Creamos una instancia del modelo
model = CategoricalNB()

# Entrenamos el modelo con los datos de entrenamiento. 
# En este paso, el algoritmo calcula las probabilidades condicionales.
model.fit(X_train, y_train)

print("¡Modelo Naive Bayes entrenado con éxito!")

¡Modelo Naive Bayes entrenado con éxito!


## Paso 6: Evaluar el Modelo

Una vez entrenado, usamos el modelo para hacer predicciones sobre el conjunto de prueba y comparamos esas predicciones con los resultados reales para medir su rendimiento.

In [13]:
# El modelo predice las etiquetas para los datos de prueba
y_pred = model.predict(X_test)

# Calculamos la precisión (accuracy)
accuracy = accuracy_score(y_test, y_pred)

print("--- Resultados de la Evaluación ---")
print(f'Precisión (Accuracy): {accuracy:.2f}')

print("Reporte de Clasificación:")
# Este reporte nos da métricas más detalladas para cada clase
print(classification_report(y_test, y_pred, zero_division=0))

--- Resultados de la Evaluación ---
Precisión (Accuracy): 0.91
Reporte de Clasificación:
                           precision    recall  f1-score   support

      Desempeño Aceptable       0.00      0.00      0.00         5
          Desempeño Bueno       0.92      0.97      0.94       229
      Desempeño Excelente       0.84      0.79      0.82        53
Desempeño insatisfactorio       1.00      0.33      0.50         6
              No evaluada       1.00      1.00      1.00        24

                 accuracy                           0.91       317
                macro avg       0.75      0.62      0.65       317
             weighted avg       0.90      0.91      0.90       317



## Paso 7: Realizar una Predicción para un Nuevo Caso

Esta es la aplicación final del modelo: usarlo para obtener información sobre un caso nuevo. Crearemos un nuevo curso y le pediremos al modelo que prediga su valoración de desempeño.

In [14]:
# Supongamos un nuevo curso con estas características
nuevo_curso = pd.DataFrame([
    {'Tipo': 'VIRTUAL', 'Tipo_Curso': 'PD', 'Plantilla': 'INNOVAME'}
])

print("--- Predicción para un Nuevo Curso ---")
print("Características del nuevo curso:")
print(nuevo_curso)
encoder.fit(X_train)
# IMPORTANTE: Debemos usar el MISMO codificador que ajustamos antes para transformar los datos nuevos
nuevo_curso_encoded = encoder.transform(nuevo_curso)

# El modelo predice la clase
prediccion_final = model.predict(nuevo_curso_encoded)

# También podemos pedirle las probabilidades que calculó para cada clase
probabilidades = model.predict_proba(nuevo_curso_encoded)

print(f'La valoración predicha para este curso es: {prediccion_final[0]}')

print("Probabilidades calculadas para cada clase:'")
prob_df = pd.DataFrame(probabilidades, columns=model.classes_)
print(prob_df.to_string(index=False))

--- Predicción para un Nuevo Curso ---
Características del nuevo curso:
      Tipo Tipo_Curso Plantilla
0  VIRTUAL         PD  INNOVAME


ValueError: Negative values in data passed to CategoricalNB (input X).

## Conclusión

En este notebook, hemos construido con éxito un clasificador Naive Bayes para predecir el desempeño docente. Hemos cubierto el ciclo de vida completo de un proyecto de Machine Learning: desde la carga y preprocesamiento de los datos, pasando por el entrenamiento y la evaluación del modelo, hasta su aplicación para hacer nuevas predicciones. Este es un ejemplo excelente de cómo una técnica clásica y eficiente como Naive Bayes puede ser aplicada para extraer valor de los datos.